In [ ]:
import numpy as np
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation
from IPython.display import HTML
font = {'size'   : 15}
matplotlib.rc('font', **font)

fft = np.fft.fft
ifft = np.fft.ifft

In this notebook we approximate the solution of the KdV-Hyperbolic (KdVH) equation 
$$
    u_t + u u_x + u_{xxx} = 0 \\
    \tau v_t - v_x + w = 0 \\
    \tau w_t + u_x - v = 0.
$$

using the HLL method.
We use this to investigate the presence of shocks and to check convergence to KdV as $\tau \to 0$.

# KdV solution via ImEx RK / Fourier pseudospectral
This code is taken directly from "PseudspectralPython".  It uses an implicit-explicit Runge-Kutta method to solve the KdV equation using a Fourier pseudospectral method.  See that repository for more details.

In [ ]:
# Higueras (17)
A    = np.array([[0,0,0],[5/6.,0,0],[11/24,11/24,0]])
Ahat = np.array([[2./11,0,0],[205/462.,2./11,0],[2033/4620,21/110,2/11]])
b = np.array([24/55.,1./5,4./11])
bhat = b

def rhs_f(u, xi, filtr):
    # Evaluate only the non-stiff nonlinear term
    uhat = np.fft.fft(u)
    return -u*np.real(np.fft.ifft(1j*xi*uhat*filtr))

def solve_KdV_ImEx2(u0,tmax,m,dt,use_filter=True,num_plots=40):
    L = 200*np.pi
    x = np.arange(-m/2,m/2)*(L/m)
    xi = np.fft.fftfreq(m)*m*2*np.pi/L

    u = u0(x)

    uhat2 = np.abs(np.fft.fft(u))

    nplt = np.floor((tmax/num_plots)/dt)
    print(nplt)
    nmax = int(round(tmax/dt))

    frames = [u.copy()]
    tt = [0]
    uuhat = [uhat2]

    filtr = np.ones_like(xi)
    
    if use_filter:
        xi_max = np.max(np.abs(xi))
        filtr[np.where(np.abs(xi)>xi_max*2./3)] = 0.

    # Assumes all diagonal entries of Ahat are equal:
    Lfactor = 1/(1-Ahat[0,0]*dt*1j*xi**3)
    D = 1j*xi**3
    
    for n in range(1,nmax+1):
        y1 = ifft(Lfactor*fft(u))
        Ly1 = ifft(D*fft(y1))
        fy1 = rhs_f(y1,xi,filtr)
        y2rhs = u + dt*(A[1,0]*fy1+Ahat[1,0]*Ly1)
        y2 = ifft(Lfactor*fft(y2rhs))
        Ly2 = ifft(D*fft(y2))
        fy2 = rhs_f(y2,xi,filtr)
        y3rhs = u + dt*(A[2,0]*fy1+A[2,1]*fy2 + Ahat[2,0]*Ly1 + \
                        Ahat[2,1]*Ly2)
        y3 = ifft(Lfactor*fft(y3rhs))
        Ly3 = ifft(D*fft(y3))
        fy3 = rhs_f(y3,xi,filtr)
        u_new = u + dt*np.real(b[0]*(fy1+Ly1) + b[1]*(fy2+Ly2) + \
                               b[2]*(fy3+Ly3))
            
        u = u_new.copy()
        t = n*dt
        # Plotting
        if np.mod(n,nplt) == 0:
            frames.append(u.copy())
            tt.append(t)
            uhat2 = np.abs(np.fft.fft(u))
            uuhat.append(uhat2)
    return x, tt, frames

# Solution of KdVH via HLL / finite volume

In [ ]:
def HLL(uvw,dtoverdx,tau):
    "Harten-Lax-van Leer Riemann solver for the KdVH system."
    eps = 1/tau
    u = uvw[0,:]
    v = uvw[1,:]
    w = uvw[2,:]
    uvwnew = np.zeros_like(uvw)
    lamda_minus = np.minimum(0.5*u[:-1]-0.25*np.sqrt(u[:-1]**2+eps),0.5*u[1:]-np.sqrt(0.25*u[1:]**2+eps))
    lamda_plus  = np.maximum(0.5*u[:-1]+0.25*np.sqrt(u[:-1]**2+eps),0.5*u[1:]+np.sqrt(0.25*u[1:]**2+eps))
    lamda_minus  = np.minimum(lamda_minus,-eps)

    # Eigenvalues at u=0:
    #lamda_minus = np.minimum(-np.sqrt(eps),-eps)
    #lamda_plus  =  np.sqrt(eps)

    # q_l and q_r
    fl1 = 0.5*u[:-1]**2 + w[:-1]
    fr1 = 0.5*u[ 1:]**2 + w[1:]
    fl2 = -eps*v[:-1]
    fr2 = -eps*v[ 1:]
    fl3 = eps*u[:-1]
    fr3 = eps*u[ 1:]
    
    # HLL Flux
    fhll1 = (lamda_plus*fl1 - lamda_minus*fr1 + lamda_plus*lamda_minus*(u[1:]-u[:-1]))/(lamda_plus-lamda_minus)
    fhll2 = (lamda_plus*fl2 - lamda_minus*fr2 + lamda_plus*lamda_minus*(v[1:]-v[:-1]))/(lamda_plus-lamda_minus)
    fhll3 = (lamda_plus*fl3 - lamda_minus*fr3 + lamda_plus*lamda_minus*(w[1:]-w[:-1]))/(lamda_plus-lamda_minus)

    fac = 1.
    uvwnew[0,1:-1] = uvw[0,1:-1] - fac*dtoverdx*(fhll1[1:]-fhll1[:-1])
    uvwnew[1,1:-1] = uvw[1,1:-1] - fac*dtoverdx*(fhll2[1:]-fhll2[:-1])
    uvwnew[2,1:-1] = uvw[2,1:-1] - fac*dtoverdx*(fhll3[1:]-fhll3[:-1])
    return uvwnew

def LxF(uvw,dtoverdx,tau):
    "Lax-Friedrichs.  Seems too dissipative."
    eps = 1/tau
    u = uvw[0,:]
    v = uvw[1,:]
    w = uvw[2,:]
    uvwnew = np.zeros_like(uvw)
    uvwnew[0,1:-1] = 0.5*(u[2:]+u[:-2]) - 0.5*dtoverdx*(u[2:]**2/2-u[:-2]**2/2+w[2:]-w[:-2])
    uvwnew[1,1:-1] = 0.5*(v[2:]+v[:-2]) + 0.5*dtoverdx*eps*(v[2:]-v[:-2])
    uvwnew[2,1:-1] = 0.5*(w[2:]+w[:-2]) - 0.5*dtoverdx*eps*(u[2:]-u[:-2])
    return uvwnew

def source(uvw,dt,tau):
    "Exact solution of the ODE for the source term."
    v = uvw[1,:]
    w = uvw[2,:]
    vnew = v*np.cos(dt/tau) - w*np.sin(dt/tau)
    wnew = v*np.sin(dt/tau) + w*np.cos(dt/tau)
    uvw[1,:] = vnew
    uvw[2,:] = wnew
    return uvw

def solve_KdVH(u0,L=100, tmax=1.,m=256, ylims=(-1,3), xlims=(-100*np.pi,100*np.pi), tau=1.0, 
               dt_factor=1., well_prepared=True,plot_vw=True, num_plots=40):
    eps = 1/tau
    # Grid
    x = np.arange(-m/2,m/2)*(L/m)

    u_max = np.max(u0(x))
    max_speed = max(eps,0.5*(u_max+np.sqrt(u_max**2+4*eps)))
    dt = 1.73*L/(m/2)/max_speed * dt_factor
    dx = x[1]-x[0]
    dtoverdx = dt/dx
    
    uvw = np.zeros((3,len(x)))
    uvw[0,:] = u0(x)
    if well_prepared:
        uvw[2,1:] = np.diff(u0(x))/dx;
        uvw[1,1:] = np.diff(uvw[2,:])/dx;
    else:
        uvw[2,1:] = 0.
        uvw[1,1:] = 0.
    

    nplt = np.floor((tmax/num_plots)/dt)
    nmax = int(round(tmax/dt))

    lw = 2
    fig = plt.figure(figsize=(12,6))
    axes = fig.add_subplot(111)
    line, = axes.plot(x,uvw[0,:],lw=lw)
    axes.set_xlabel(r'$x$',fontsize=30)
    if plot_vw:
        #axes2 = fig.add_subplot(312)
        vline, = axes.plot(x,uvw[1,:],lw=lw)
        #axes3 = fig.add_subplot(313)
        wline, = axes.plot(x,uvw[2,:],lw=lw)
    plt.close()

    frames = [uvw.copy()]
    tt = [0]

    for n in range(1,nmax+1):
        uvw_star = HLL(uvw,dtoverdx,tau)
        uvw_new = source(uvw_star,dt,tau)

        uvw = uvw_new.copy()
        t = n*dt
        # Plotting
        if np.mod(n,nplt) == 0:
            frames.append(uvw.copy())
            tt.append(t)
        
    def plot_frame(i):
        line.set_data(x,frames[i][0,:])
        if plot_vw:
            vline.set_data(x,frames[i][1,:])
            wline.set_data(x,frames[i][2,:])
        axes.set_title('t= %.2e' % tt[i])
        axes.set_xlim(xlims)
        axes.set_ylim(ylims)
        if plot_vw:
            axes.legend(['$u$', '$v$', '$w$'], loc='upper right', fontsize=20)

    anim = matplotlib.animation.FuncAnimation(fig, plot_frame,
                                       frames=len(frames), interval=100)

    return HTML(anim.to_jshtml()), x, tt, frames

# Checking convergence

In [ ]:
tau = 1.0

def u0(x):
    return 2*np.exp(-x**2/50)

x, tt, frames = solve_KdV_ImEx2(u0, tmax=5., m=2048, dt=1e-3, use_filter=True)
anim, x10, tt, frames10 = solve_KdVH(u0, tmax=5., ylims=(-1,3.5), xlims=(-50,50), m=8000, tau=1.0, dt_factor=0.1)
anim, x05, tt, frames05 = solve_KdVH(u0, tmax=5., ylims=(-1,3.5), xlims=(-50,50), m=8000, tau=0.5, dt_factor=0.1)
anim, x01, tt, frames01 = solve_KdVH(u0, tmax=5., ylims=(-1,3.5), xlims=(-50,50), m=8000, tau=0.1, dt_factor=0.1)

In [ ]:
plt.plot(x, frames[-1],label='KdV')
plt.plot(x10,frames10[-1][0,:],label=r'$\tau=1$')
plt.plot(x05,frames05[-1][0,:],label=r'$\tau=0.5$')
plt.plot(x01,frames01[-1][0,:],label=r'$\tau=0.1$')

plt.xlim(-30,30)
plt.legend()
plt.xlabel('x')
plt.ylabel('u')
plt.title('t=5')
plt.savefig('Convergence.png',dpi=300)

# Experiments
Here we start with a Gaussian and test whether shocks form, depending on the value of $\tau$.

In [ ]:
tau = 1.0

def u0(x):
    return 2*np.exp(-x**2/50)

anim, x10, tt, frames10 = solve_KdVH(u0, tmax=40., ylims=(-1,3.5), xlims=(-50,50),
                                     m=8000, tau=tau, dt_factor=0.1)
anim

In [ ]:
tau = 0.5

anim, x05, tt, frames05 = solve_KdVH(u0, tmax=40., ylims=(-1,3.5), xlims=(-50,50),
                                     m=8000, tau=tau, dt_factor=0.1)
anim

In [ ]:
tau = 0.1

anim, x01, tt, frames01 = solve_KdVH(u0, tmax=40., ylims=(-1,3.5), xlims=(-50,50),
                                     m=8000, tau=tau, dt_factor=0.1)
anim

In [ ]:
x, tt, frames = solve_KdV_ImEx2(u0, tmax=40., m=2048, dt=1e-3, use_filter=True)

In [ ]:
iframe = 16
xlim = (-35,35)
fig, axes = plt.subplots(1,3, figsize=(18,5))
axes[0].plot(x10,frames10[iframe][0,:])
axes[0].set_xlim(xlim)
axes[0].set_title("$\\tau=1$")
axes[1].plot(x05,frames05[iframe][0,:])
axes[1].set_xlim(xlim)
axes[1].set_title("$\\tau=0.5$")
axes[2].plot(x01,frames01[iframe][0,:])
axes[2].set_xlim(xlim)
axes[2].set_title("$\\tau=0.1$")
axes[2].plot(x,frames[iframe], 'k--', label='KdV')
plt.savefig('KdVH_solutions.png', dpi=300)

In [ ]:
iframe = 25
fig, axes = plt.subplots(1,3, figsize=(18,5))
axes[0].plot(x10,frames10[iframe][0,:])
axes[0].set_xlim(-50,50)
axes[0].set_title("$\\tau=1$")
axes[1].plot(x05,frames05[iframe][0,:])
axes[1].set_xlim(-50,50)
axes[1].set_title("$\\tau=0.5$")
axes[2].plot(x01,frames01[iframe][0,:])
axes[2].set_xlim(-50,50)
axes[2].set_title("$\\tau=0.1$")